In [1]:
import numpy as np
import sympy
from sympy import lambdify
np.set_printoptions(suppress=True)
np.set_printoptions(precision=4)

* y
* y = 2x + 1
* y = -2x + 10
* y = 3x + 1
* y = -3x + 15

In [2]:
# x = np.array([[1, 2, 3, 4, 5], [3, 5, 7, 9, 11], [8, 6, 4, 2, 0], [4, 7, 10, 13, 16], [12, 9, 6, 3, 0]])

In [3]:
def generate_matrix_with_dominant_pc(variance_threshold_L = 0.8, variance_threshold_U = 1):
    """Generates a random 5x5 symmetric matrix with positive values and
       the first PC explaining at least 90% of the variance.
    """

    while True:
        # Create a nxn array with random positive values
        matrix = np.abs(np.random.randn(6, 6) * 10)  # Use absolute values for positive entries
        # Make the array symmetric
        matrix = matrix + matrix.T
        
        origin_matrix = matrix
        # Subtract the mean of each row to center the data
        matrix = matrix - matrix.mean(axis=1, keepdims=True)

        # SVD 
        n = len(matrix[0])
        matrix_t = np.transpose(matrix)
        y = matrix_t / np.sqrt(n)
        U, S, Vh = np.linalg.svd(y, full_matrices=True)
        eigenvalues = S * S
        EV1_explained_variance = eigenvalues[0] / np.sum(eigenvalues) 

        # Check if the first PC explains at least the desired variance
        if EV1_explained_variance >= variance_threshold_L and EV1_explained_variance < variance_threshold_U:
            print(origin_matrix, '\n')
            print(matrix, '\n')
            return matrix

# Generate a matrix that meets the conditions
x = generate_matrix_with_dominant_pc()

[[50.7963 12.734  19.0897 11.3389 15.7492 40.4103]
 [12.734  20.1235 19.6874 30.5553 18.4224 17.7738]
 [19.0897 19.6874 15.4655 17.8718 13.6019 27.0883]
 [11.3389 30.5553 17.8718 23.3893 21.2684  6.6252]
 [15.7492 18.4224 13.6019 21.2684 24.5885 13.4656]
 [40.4103 17.7738 27.0883  6.6252 13.4656 21.7452]] 

[[ 25.7765 -12.2857  -5.93   -13.6808  -9.2706  15.3905]
 [ -7.1487   0.2408  -0.1953  10.6726  -1.4603  -2.1089]
 [  0.289    0.8866  -3.3353  -0.929   -5.1989   8.2875]
 [ -7.1692  12.0471  -0.6364   4.8812   2.7603 -11.8829]
 [ -2.1002   0.5731  -4.2474   3.4191   6.7391  -4.3837]
 [ 19.2255  -3.411    5.9036 -14.5595  -7.7191   0.5605]] 



In [4]:
corr_x = np.corrcoef(x)
display(corr_x)

array([[ 1.    , -0.7263,  0.5329, -0.8522, -0.5793,  0.8081],
       [-0.7263,  1.    , -0.1538,  0.5058,  0.3977, -0.8284],
       [ 0.5329, -0.1538,  1.    , -0.5323, -0.583 ,  0.1505],
       [-0.8522,  0.5058, -0.5323,  1.    ,  0.5855, -0.5587],
       [-0.5793,  0.3977, -0.583 ,  0.5855,  1.    , -0.6574],
       [ 0.8081, -0.8284,  0.1505, -0.5587, -0.6574,  1.    ]])

### PCA

In [5]:
arr = [x]
EV1 = []

for target_x in arr:
    n = len(target_x[0])

    x_t = np.transpose(target_x)
    y = x_t / np.sqrt(n)

    U, S, Vh = np.linalg.svd(y, full_matrices=True)
    eigenvalues = S * S
    sum_eigenvalues = np.sum(eigenvalues)
    explained_variances = eigenvalues / sum_eigenvalues
    print(np.round(explained_variances, 4), '\n')

    cov_x = np.matmul(np.transpose(y), y)
    V_Cx = np.matmul(Vh, cov_x)
    
    # display(np.matmul(V_Cx, np.transpose(Vh)))

    ## EV1
    EV1 = (Vh[0])
    
    cov_x_GaussForm = []
    add_col = []
    for i in range(len(cov_x[0])):
        add_col.append([np.dot(Vh[-1], cov_x[i])])

    add_col = np.array(add_col)

    cov_x_GaussForm = np.concatenate((cov_x, add_col), axis=1)
    cov_x_rref = sympy.Matrix(cov_x_GaussForm).rref()
    Base_Vec = []
    
    for i in cov_x_rref[0]:
        if i != 0 and i != 1:
            Base_Vec.append(i)
    
    # print(Vh[-1] / Base_Vec)
    # print(Base_Vec)
    # print(Vh[-1])
    # print('\n')
    # print(cov_x)

    # print(np.dot(EV1, Base_Vec))

    # display(corr_x > 0)
    # display(Vh > 0)
    # display(V_Cx > 0)
    # display((Vh > 0) == (V_Cx > 0))

    print(EV1, '\n')
    print(cov_x, '\n')
    
    cov_x_sum = [np.sum(np.abs(row)) for row in cov_x]
    print(cov_x_sum, '\n') 

[0.8198 0.106  0.0338 0.0251 0.0154 0.    ] 

[-0.7591  0.2152 -0.1026  0.3373  0.1366 -0.4843] 

[[ 226.7516  -58.4997   34.1316 -100.7141  -35.4513  130.3062]
 [ -58.4997   28.6141   -3.5003   21.2333    8.6459  -47.4516]
 [  34.1316   -3.5003   18.0947  -17.7722  -10.0791    6.8571]
 [-100.7141   21.2333  -17.7722   61.5976   18.6743  -46.9526]
 [ -35.4513    8.6459  -10.0791   18.6743   16.5172  -28.6107]
 [ 130.3062  -47.4516    6.8571  -46.9526  -28.6107  114.6642]] 

[585.8544905331183, 167.94484275919993, 90.43492765046679, 266.9440571557788, 117.97847715511412, 374.842353438763] 



Test-V2: Confirm the reltionship between the EV1's pos-neg pattern and the corresponding Covariance matrix row.

If the explained variance is high enough, the EV1's pos-neg pattern will match the pos-neg pattern of the Covariance matrix with maximal absolute value sum of that row (compare to every row of the covariance matirx)